<a href="https://colab.research.google.com/github/j-river1/TFM/blob/master/reconstructing_brain_images_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Reconstructing Brain MRI Images Using Deep Learning (Convolutional Autoencoder)**

https://www.datacamp.com/community/tutorials/reconstructing-brain-images-deep-learning

In this tutorial, you'll learn & understand how to read nifti format brain magnetic resonance imaging (MRI) images, reconstructing them using convolutional autoencoder.
You will use 3T brain MRI dataset to train your network. To observe the effectiveness of your model, you will be testing your model on :

1. Unseen 3T MRI images,
2. Noisy 3T MRI images and
3. Use a qualitative metric: Peak signal to noise ratio (PSNR) to evaluate the performance of the reconstructed images.


This tutorial will not be addressing the intricacies of medical imaging but will be focused on the deep learning side! Note : This tutorial will mostly cover the practical implementation of convolutional autoencoders. So, if you are not yet aware about **convolutional neural network (CNN) and autoencoder**, you might want to look at CNN and Autoencoder tutorial. The best part about this tutorial is that you will be loading the 3D volumes as 2D images and will be feeding them in to the model. In a nutshull, you will address the following topics in today's tutorial:

In the beginning you will be briefed about Magnetic Resonance Imaging (MRI),
Then you will understand about the Brain MRI dataset: what kind of images it has, importing the modules, how to read the images, creating an array of the images, preprocessing the brain MRI images to be able to feed them in the model and finally explore the brain MRI images.

In the implementation of convolutional autoencoder: you will Fit the preprocessed data into the model, visualize the training and validation loss plot, sabe the trained model and finally predict on the test set.

Next, you'll will test the Robustness of your pre-trained model by adding noise into the test images and see how well the model performs quantitatively.

Finally, you will test your predictions using a quantitative metric peak signal-to-noise ratio (PSNR) and measure the performance of your model.

**Brief Introduction on MR images**


A variety of systems are used in medical imaging ranging from open MRI units with magnetic field strength of 0.3 Tesla (T) to extremity MRI systems with field strengths up to 1.0 T and whole-body scanners with field strengths up to 3.0 T (in clinical use). Tesla is the unit of measuring the quantitative strength of magnetic field of MR images. High field MR scanners (7T, 11.5T) yielding higher SNR (signal-to-noise ratio) even with smaller voxel (a 3-dimensional patch or a grid) size and are thus preferred for more accurate diagnosis.

A smaller voxel size leads to a better resolution, which can in turn aid clinical diagnosis. However, the strength of magnetic field being used in MR scanner puts a lower bound on voxel size to maintain a good signal to noise ratio (SNR), in order to preserve the MR image details.
Despite the superior image quality of 7T and 11.5T, they are rarely deployed in production due to its cost constraints.

According to the recent papers, the reported number of 3T scanners are ~20,000 compared to just ~40 7T scanners.

**Understanding the Brain MRI 3T Dataset**

The brain MRI dataset consists of 3D volumes each volume has in total 207 slices/images of brain MRI's taken at different slices of the brain. Each slice is of dimension 173 x 173. The images are single channel grayscale images. There are in total 30 subjects, each subject containing the MRI scan of a patient. The image format is not jpeg,png etc. but rather nifti format. You will see in later section how to read the nifti format images.

The dataset consists of T1 modality MR images, T1 sequences are traditionally considered good for evaluation of anatomic structures. The dataset on which you will be working today consists of 3T Brain MRI's.

The dataset is public and is available for download at this source.

Tip: if you want to learn how to implement an Multi-Layer Perceptron (MLP) for classification tasks with the MNIST dataset, check out this tutorial.

Note : Before you begin please note that the model will be trained on a system with Nvidia 1080 Ti GPU Xeon e5 GeForce processor with 32GB RAM. If you are using Jupyter Notebook, you will need to add three more lines of code where you specify CUDA device order and CUDA visible devices using a module called os.

In the code below, you basically set environment variables in the notebook using os.environ. It's good to do the following before initializing Keras to limit Keras backend TensorFlow to use first GPU. If the machine on which you train on has a GPU on 0, make sure to use 0 instead of 1. You can check that by running a simple command on your terminal: for example, nvidia-smi

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1" #model will be trained on GPU 1

**Importing the modules**

First, you import all the required modules like cv2, numpy, matplotlib and most importantly keras, since you'll be using that framework in today's tutorial!
In order to read the nifti format images, you also have to import a module called nibabel.

In [ ]:
import os
import cv2
from keras.layers import Input,Dense,Flatten,Dropout,merge,Reshape,Conv2D,MaxPooling2D,UpSampling2D,Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import regularizers
from keras import backend as K

In [ ]:
import numpy as np
import scipy.misc
import numpy.random as rng
from PIL import Image, ImageDraw, ImageFont
from sklearn.utils import shuffle
import nibabel as nib #reading MR images
from sklearn.model_selection import train_test_split
import math
import glob
from matplotlib import pyplot as plt
%matplotlib inline

**Loading the data**

You will use glob module which will return a list comprising of all the volumes in the folder that you specify!

In [ ]:
ff = glob.glob('/content/drive/My Drive/Colab Notebooks/TFM/Datos/*')


Let's print the first element of the list and also check the length of the list: which in our case should be 30.

In [ ]:
ff[0]

'/content/drive/My Drive/Colab Notebooks/TFM/Datos/z_RF_448_1.nii.gz'

Let's print the first element of the list and also check the length of the list: which in our case should be 1

In [ ]:
len(ff)

1

Now you are all set to load the 3D volumes using nibabel. Note that when you load a Nifti format volume, Nibabel does not load the image array. It waits until you ask for the array data. The normal way to ask for the array data is to call the get_data() method.

Since you want the 2D slices instead of 3D, you will initialise a list in which; every time you read a volume, you will iterate over all the complete 207 slices of the 3D volume and append each slice one by one in to a list.

In [ ]:
images = []

Let's also print the shape of one of the 3D volume, it should be of the shape 173 x 207 x 173 (x, y, z; coordinates).

Note : You will be using only the middle 51 slices of the brain and not all the 207 slices. So, let's also see how to use only the center slices and load them

In [ ]:
for f in range(len(ff)):
    a = nib.load(ff[f])
    a = a.get_data()
    a = a[:,78:129,:]
    for i in range(a.shape[1]):
        images.append((a[:,i,:]))
print (a.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  This is separate from the ipykernel package so we can avoid doing imports until


(91, 31, 91)


Let's analyse shape of one slice out of 207 slices.

In [ ]:
a[:,0,:].shape

(91, 91)